In [ ]:
dirc_path = '/home/habjan.e/'

import sys
sys.path.append(dirc_path + "TNG/TNG_cluster_dynamics")
import TNG_DA
import numpy as np 
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from astropy.io import fits
from astropy.table import Table

import tensorflow_datasets as tfds
import tensorflow as tf

import os
sys.path.append(os.getcwd())
from training_structure import train_model, create_dataloader

In [ ]:
import multiprocessing as mp

In [ ]:
pro_num = os.cpu_count()
pool = mp.Pool(processes = pro_num)
split = pro_num

batch = math.ceil(len(train)/split)
train_list = [train[i:i+batch] for i in range(0, len(train), batch)]
param_train_list = [(train_list[i], 'train') for i in range(len(train_list))]
train_processes = [pool.apply_async(main, args = p) for p in param_train_list]

batch = math.ceil(len(test)/split)
test_list = [test[i:i+batch] for i in range(0, len(test), batch)]
param_test_list = [(test_list[i], 'test') for i in range(len(test_list))]
test_processes = [pool.apply_async(main, args = p) for p in param_test_list]